In [119]:
import openai
from utils import load_api_key

In [120]:
openai.api_key = load_api_key("/Users/gursi/Desktop/openai-api.txt")

In [122]:
def note_generator(chat_completion: openai.ChatCompletion, outline: dict[str, list[str]], level: str, subject: str):
    prompt = engineer_prompt_note_gen(outline, level, subject)
    model_output = chat_completion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content":f"Your are a {level} {subject} notes generator."},
            {"role": "user", "content":prompt}
        ]
    )
    model_output = parse_results(model_output)
    return model_output
        
    
def engineer_prompt_note_gen(outline: dict[str, list[str]], level: str, subject: str):
    prompt = f'Generate quick revision notes for me on the following topics for a {subject} course on a {level} level. Include all equations. Explain each variable and concept. Do not say "sure, here are some notes..".\n\n'
    for topic in outline:
        prompt += f"- {topic}\n"
        if outline[topic] is not None:
            for subtopic in outline[topic]:
                prompt += f"    - {subtopic}\n"
    return prompt


def parse_results(chatgpt_output: dict) -> str:
    return chatgpt_output["choices"][0]["message"]["content"]

In [123]:
openai.api_key = load_api_key("/Users/gursi/Desktop/openai-api.txt")
chat_complete = openai.ChatCompletion

In [124]:
outline = {
    "Electromagnetism":None,
    "Electricity":[
        "Resistance",
        "Energy and Power",
        "Kirchoff Laws",
        "Lenz laws"
    ],
    "Mechanics": [
        "SUVAT equations",
        "Terminal velocity",
        "Friction and Drag"
    ]
}
output = note_generator(chat_complete, outline, "High school", "Physics")

In [125]:
print(output)

Electromagnetism:

- Electromagnetism is the branch of physics responsible for the study of the properties and interactions of electric currents and magnetic fields.
- The electromagnetic force is one of the four fundamental forces of the universe and is responsible for the electric and magnetic fields that exist around us.

Electricity:

Resistance:

- Resistance is the measure of obstruction of the flow of electric current through a circuit. 
- The unit of resistance is ohms (Ω). 
- The relationship between resistance, current, and potential difference is given by Ohm's Law as V = IR, where V is the potential difference, I is the current, and R is the resistance.

Energy and Power:

- Energy is the ability to do work, and is measured in joules (J).
- Power is the rate at which work is done, and is measured in watts (W). 
- The relationship between energy, power, and time is given by P = E/t, where P is power, E is energy, and t is time.

Kirchoff Laws:

- Kirchoff's laws are two fund

In [27]:
from PyPDF2 import PdfReader
from utils import parse_results
from sklearn.cluster import KMeans
import numpy as np
import openai
import string

In [2]:
def parse_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text_body = ""
    for page in reader.pages:
        text_body += " " + page.extract_text()
    return text_body


def extract_keywords_from_prompt(chat_completion: openai.ChatCompletion, prompt: str):
    model_output = chat_completion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content":f"Your job is to extract key words from text. Generic words should never be extracted, only topic specific words."},
            {"role": "user", "content":f"Extract the keywords from the following instruction. Output a single list of comma separated values only once. \n\n {prompt}"}
        ]
    )
    output = parse_results(model_output)
    return output.split(",")

In [37]:
text_body = parse_pdf("/Users/gursi/Desktop/nst.pdf")
max_num_words = 200
prompt = "Summarize the loss function, optimization method and model architecture used in this study for me."
chat_completion = openai.ChatCompletion
buffer = 400

# keywords = extract_keywords_from_prompt(chat_completion, prompt)
keywords = ['loss function', ' optimization method', ' model architecture', ' study.']
keywords = [k.translate(str.maketrans('', '', string.punctuation)).strip().lower() for k in keywords]
kw = []
[[kw.append(w) for w in word.split(" ")] for word in keywords]
arr_text = np.array(text_body.lower().split())

idxs = np.array([])
max_idx = len(arr_text)
for keyw in kw:
    kw_idxs = np.where(arr_text == keyw)[0] / max_idx
    idxs = np.concatenate([idxs, kw_idxs])

kmeans = KMeans(n_clusters = len(keywords))
labels = kmeans.fit_predict(idxs.reshape(-1, 1))
centroid_idxs = list((kmeans.cluster_centers_ * len(arr_text)).astype(int).reshape(-1))

for centroid_idx in centroid_idxs:
    


In [40]:
i = centroid_idxs.reshape(-1)[0]
arr_text[max(0, i - buffer):min(len(arr_text, ))]

'encoded'

In [26]:
idxs

array([0.35428006, 0.63854527, 0.64398812, 0.70781791, 0.71128154,
       0.71622959, 0.76224641, 0.78698664, 0.35452746, 0.76249381,
       0.49059871, 0.58881742, 0.89658585, 0.99430975, 0.49505195])